In [ ]:
# lsfdemo - Program for demonstrating least squares fit routines

# Set up configuration options and special features
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def linreg(x,y,sigma):
    # Function to perform linear regression (fit a line)
    # Inputs
    #   x       Independent variable
    #   y       Dependent variable
    #   sigma   Estimated error in y
    # Outputs
    #   a_fit   Fit parameters; a(1) is intercept, a(2) is slope
    #   sig_a   Estimated error in the parameters a()
    #   yy      Curve fit to the data
    #   chisqr  Chi squared statistic

    #* Evaluate various sigma sums
    s = 0; sx = 0; sy = 0; sxy = 0; sxx = 0
    for i in range(len(x)):
        sigmaTerm = sigma[i]**(-2)
        s += sigmaTerm              
        sx += x[i] * sigmaTerm
        sy += y[i] * sigmaTerm
        sxy += x[i] * y[i] * sigmaTerm
        sxx += x[i]**2 * sigmaTerm
    denom = s*sxx - sx**2

    #* Compute intercept a_fit(1) and slope a_fit(2)
    a_fit = np.empty(2)
    a_fit[0] = (sxx*sy - sx*sxy)/denom
    a_fit[1] = (s*sxy - sx*sy)/denom

    #* Compute error bars for intercept and slope
    sig_a = np.empty(2)
    sig_a[0] = np.sqrt(sxx/denom)
    sig_a[1] = np.sqrt(s/denom)

    #* Evaluate curve fit at each data point and compute Chi^2
    yy = np.empty(len(x))
    chisqr = 0.
    for i in range(len(x)):
        yy[i] = a_fit[0]+a_fit[1]*x[i]          # Curve fit to the data
        chisqr += ((y[i]-yy[i])/sigma[i])**2    # Chi square
    return [a_fit, sig_a, yy, chisqr]

In [ ]:
def pollsf(x, y, sigma, M):
    # Function to fit a polynomial to data
    # Inputs 
    #   x       Independent variable
    #   y       Dependent variable
    #   sigma   Estimate error in y
    #   M       Number of parameters used to fit data
    # Outputs
    #   a_fit   Fit parameters; a(1) is intercept, a(2) is slope
    #   sig_a   Estimated error in the parameters a()
    #   yy      Curve fit to the data
    #   chisqr  Chi squared statistic

    #* Form the vector b and design matrix A   
    N = len(x)
    b = np.empty(N)
    A = np.empty((N,M))
    for i in range(N):
        b[i] = y[i]/sigma[i]
        for j in range(M):
            A[i,j] = x[i]**j / sigma[i] 

    #* Compute the correlation matrix C 
    C = np.linalg.inv( np.dot( np.transpose(A), A) )

    #* Compute the least squares polynomial coefficients a_fit
    a_fit = np.dot(C, np.dot( np.transpose(A), np.transpose(b)) )

    #* Compute the estimated error bars for the coefficients
    sig_a = np.empty(M)
    for j in range(M):
        sig_a[j] = np.sqrt(C[j,j])

    #* Evaluate curve fit at each data point and compute Chi^2
    yy = np.zeros(N)
    chisqr = 0.
    for i in range(N):
        for j in range(M):
            yy[i] += a_fit[j]*x[i]**j   # yy is the curve fit
        chisqr += ((y[i]-yy[i]) / sigma[i])**2
        
    return [a_fit, sig_a, yy, chisqr]

In [ ]:
#* Initialize data to be fit. Data is quadratic plus random number.
print 'Curve fit data is created using the quadratic'
print '  y(x) = c(0) + c(1)*x + c(2)*x**2'
c = np.array(input('Enter the coefficients as [c(0) c(1) c(2)]: '))
N = 50;                 # Number of data points
x = np.arange(1,N+1)    # x = [1, 2, ..., N]
y = np.empty(N)
sigma = np.empty(N)
np.random.seed(0)       # Initialize random number generator
alpha = input('Enter estimated error bar: ')
for i in range(N):
    r = alpha * np.random.normal()    # Gaussian distributed random vector
    y[i] = c[0] + c[1]*x[i] + c[2]*x[i]**2 + r
    sigma[i] = alpha       # Constant error bar

In [ ]:
#* Fit the data to a straight line or a more general polynomial
M = input('Enter number of fit parameters (=2 for line): ')
if M == 2 :  
    #* Linear regression (Straight line) fit
    [a_fit, sig_a, yy, chisqr] = linreg(x,y,sigma)
else: 
    #* Polynomial fit
    [a_fit, sig_a, yy, chisqr] = pollsf(x,y,sigma,M)

In [ ]:
#* Print out the fit parameters, including their error bars.
print 'Fit parameters:'
for i in range(M):
    print ' a[', i, '] = ', a_fit[i], ' +/- ', sig_a[i]

#* Graph the data, with error bars, and fitting function.
plt.errorbar(x,y,sigma,None,'o')   # Graph data with error bars
plt.plot(x,yy,'-')            # Plot the fit on same graph as data
plt.xlabel('x_i')  
plt.ylabel('y_i and Y(x)') 
plt.title([r"\chi^2 = ",chisqr,'    N-M = ',N-M])